In [2]:
import networkx as nx
import np
from pgmpy.models import DiscreteBayesianNetwork

email_model = DiscreteBayesianNetwork([('S', 'O'), ('S', 'L'), ('S', 'M'), ('L', 'M')])
pos = nx.circular_layout(email_model)
nx.draw(email_model, with_labels=True, pos=pos)

AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.

In [ ]:
email_model.local_independencies(['S', 'L', 'M', 'O'])

In [ ]:
from pgmpy.factors.discrete import TabularCPD

# Root of the bayesian model

cpd_s = TabularCPD(variable='S', variable_card=2, values=[
    [0.6], #P(S=0)
    [0.4]  #P(S=1)
])

# Nodes of the bayesian model with 1 parent
cpd_l = TabularCPD(variable='L', variable_card=2, values=[
    [0.9, 0.3], # P(O=0|S=0), P(O=0|S=1)
    [0.1, 0.7] # P(O=1|S=0), P(O=1|S=1)
], evidence=["S"], evidence_card=[2])
cpd_o = TabularCPD(variable='O', variable_card=2, values=[
    [0.7, 0.2], # P(L=0|S=0), P(L=0|S=1)
    [0.3, 0.8] # P(L=1|S=0), P(L=1|S=1)
], evidence=["S"], evidence_card=[2])

# Nodes of the bayesian model with 2 parents
cpd_m = TabularCPD(variable='M', variable_card=2, values=[
    [0.8, 0.4, 0.5, 0.1], # P(M=0|S=0, L=0); P(M=0|S=0, L=1); P(M=0|S=1, L=0); P(M=0|S=1, L=1)
    [0.2, 0.6, 0.5, 0.9] # P(M=1|S=0, L=0); P(M=1|S=0, L=1); P(M=1|S=1, L=0); P(M=1|S=1, L=1)
], evidence=["S", "L"], evidence_card=[2, 2])

email_model.add_cpds(cpd_s, cpd_o, cpd_l, cpd_m)

print(email_model.check_model())

In [ ]:
from pgmpy.inference import VariableElimination

infer = VariableElimination(email_model)

In [ ]:


print("="*60)
print("Example 1: O=1, L=1, M=1 (has offer, has links, is long)")
print("="*60)
result1 = infer.query(['S'], evidence={'O': 1, 'L': 1, 'M': 1})
print(result1)
print(f"\nProbability of SPAM: {result1.values[1]:.4f}")
print(f"Classification: {'SPAM' if result1.values[1] > 0.5 else 'NOT SPAM'}")

In [ ]:
print("\n" + "="*60)
print("Example 2: O=0, L=0, M=0 (no offer, no links, short)")
print("="*60)
result2 = infer.query(['S'], evidence={'O': 0, 'L': 0, 'M': 0})
print(result2)
print(f"\nProbability of SPAM: {result2.values[1]:.4f}")
print(f"Classification: {'SPAM' if result2.values[1] > 0.5 else 'NOT SPAM'}")

In [ ]:
print("\n" + "="*60)
print("Example 3: O=1, L=0, M=0 (has offer, no links, short)")
print("="*60)
result3 = infer.query(['S'], evidence={'O': 1, 'L': 0, 'M': 0})
print(result3)
print(f"\nProbability of SPAM: {result3.values[1]:.4f}")
print(f"Classification: {'SPAM' if result3.values[1] > 0.5 else 'NOT SPAM'}")

In [ ]:
print("\n" + "="*60)
print("Example 4: L=1 (only know it has links)")
print("="*60)
result4 = infer.query(['S'], evidence={'L': 1})
print(result4)
print(f"\nProbability of SPAM: {result4.values[1]:.4f}")
print(f"Classification: {'SPAM' if result4.values[1] > 0.5 else 'NOT SPAM'}")

In [ ]:
urn_model = DiscreteBayesianNetwork([('D', 'A'), ('A', 'B')])

In [ ]:

# Die roll cpd. We have 6 outcomes for each face.
# Root of the bayesian model
cpd_die = TabularCPD(variable='D', variable_card=6, values=[
    [1/6],[1/6],[1/6],[1/6],[1/6],[1/6] # probability of each face is 1/6
])

# Ball adding given the die roll D
# Node with 1 parent

cpd_add = TabularCPD(variable='A', variable_card=3, values=[
    [0, 0, 0, 0, 0, 1], # If given number is 6
    [1, 0, 0, 1, 0, 0], # If given number is not prime
    [0, 1, 1, 0, 1, 0] # If given number is prime
], evidence=['D'], evidence_card=[6])

cpd_draw = TabularCPD(variable='B', variable_card=3,values=[
        [4/10, 3/10, 3/10],  # P(B=red | A) - after adding red/blue/black
        [4/10, 5/10, 4/10],  # P(B=blue | A)
        [2/10, 2/10, 3/10]   # P(B=black | A)
    ],
    evidence=['A'],
    evidence_card=[3]
)

In [ ]:
urn_model.add_cpds(cpd_die, cpd_add, cpd_draw)

In [ ]:
print(urn_model.check_model())

In [ ]:
infer = VariableElimination(urn_model)

In [ ]:
result = infer.query(['B'])
print("\nProbability distribution of drawn ball:")
print(result)
print(f"\nProbability of drawing RED ball: {result.values[0]:.4f}")
print(f"Probability of drawing BLUE ball: {result.values[1]:.4f}")
print(f"Probability of drawing BLACK ball: {result.values[2]:.4f}")

In [ ]:
pos = nx.circular_layout(urn_model)
nx.draw(urn_model, with_labels=True, pos=pos)

In [ ]:
def game_simulation():
    starter = np.random.choice([0, 1])

    n = np.random.randint(1, 6)

    if starter == 0:
        p_heads = 4/7
        m = sum(1 for _ in range(2 * n) if np.random.random() < p_heads)
        winner = 0 if n >= m else 1
    else:
        p_heads = 1/2
        m = sum(1 for _ in range(2 * n) if np.random.random() < p_heads)
        winner = 1 if n >= m else 0

    return starter, winner

In [ ]:
nr_simulatons = 10000

result = [game_simulation() for _ in range(nr_simulatons)]

p0_wins = sum(1 for _,winner in result if winner == 0)
p1_wins = sum(1 for _,winner in result if winner == 1)

p0_prob = p0_wins / nr_simulatons
p1_prob = p1_wins / nr_simulatons

print(f"Number of games won by P0: {p0_wins} Prob: {p0_prob}")
print(f"Number of games won by P1: {p1_wins} Prob: {p1_prob}")

if p0_prob > p1_prob:
    print("P0 is most likely to win")
else:
    print("P1 is most likely to win")

In [ ]:
model = DiscreteBayesianNetwork([
    ('DieRoll', 'Heads'),
    ('Starter', 'Heads')
])

In [ ]:
cpd_starter = TabularCPD(
    variable='Starter',
    variable_card=2,
    values=[[0.5], [0.5]],
    state_names={'Starter': ['P0', 'P1']}
)

cpd_die = TabularCPD(
    variable='DieRoll',
    variable_card=6,
    values=[[1/6],[1/6],[1/6],[1/6],[1/6],[1/6]],
    state_names={'n': ['1', '2', '3', '4', '5', '6']}
)

In [ ]:
from scipy.stats import binom

cpd_head_table = []

for start in ['P0', 'P1']:
    for n_val in range(1, 7):
        flips = 2*n_val
        p_head = 4/7 if start == 'P0' else 0.5
        for m_val in range(flips+1):
            prob = binom.pmf(m_val, flips, p_head)
            cpd_head_table.append([start, str(n_val), m_val, prob])

In [ ]:
cpd_heads = TabularCPD(
    variable='Heads',
    variable_card=11,
    values=[row[3] for row in cpd_head_table],
    evidence=['DieRoll', 'Starter'],
    evidence_card=[6, 2],
    state_names={'m': list(range(11)), 'Start': ['P0', 'P1'], 'n': ['1', '2', '3', '4', '5', '6']}
)

model.add_cpds(cpd_starter, cpd_die, cpd_heads)

In [ ]:
infer=VariableElimination(model)

result = infer.query(
    variables=['Starter'],
    evidence={'Heads': 1}
)

print(result)